## 到期收益率和即期利率相互转换

In [2]:
import numpy as np

In [3]:
def spot_to_par(
        self,
        Spot=None,#即期利率arrary
        Tenor=None,#期限arrary
        # Type="DISCRETE",
        ):
    assert len(self.Tenor)==len(self.Spot)
    principal_value=1/(1+self.Spot/100)**self.Tenor#不付息债，按照直接按照spot贴现
    interest=np.ceil(self.Tenor)-self.Tenor#应计利息
    # sum from 1 to n, 1/(1+st)^t
    def _coupon_df(_tenor):
        # 每个付息时间点
        # 一定要round，不然生成数列会有很低位的小数，和self.Tenor对不上
        # _tenor=self.Tenor[55]
        tenor_list=np.arange(_tenor,0,-1)[::-1].round(5)
        temp=np.isin(self.Tenor,tenor_list)
        spot_list=self.Spot[temp]/100
        output=np.sum(1/(1+spot_list)**tenor_list)#全价
        return output
    # 将self.Tenor升一个维度，方便用apply
    coupon_df=np.apply_along_axis(_coupon_df,axis=1,arr=self.Tenor.reshape(-1,1))

    # 算法更精确，目标是让spot曲线贴现的全价和par曲线贴现的全价直接相等，
    # 一般差异不大，0-10Y平均误差0.3BP，短端差异可达2BP
    # def func(c):
    #     return c*coupon_df+principal_value-(1+c)**interest
    # coupon=scipy.optimize.fsolve(func,np.empty(len(self.Tenor)))*100

    # 目标是让用spot贴现的净价始终等于1，净价=全价-t/365*c，应计利息不贴现
    coupon=(1-principal_value)/(coupon_df-interest)*100
    self.Par=coupon
    return self

In [186]:
def par_to_spot(
        Par=None,
        Tenor=None ):
    
    assert len(Par) == len(Tenor)
    Par = Par/100 
    Tenor = Tenor.round(5)#取近似避免无法匹配
    interest=(np.ceil(Tenor)-Tenor)*Par#应计利息
    discounted_factor = 1/(1+Par)**Tenor #不同spot下的折旧因子，把初始值先设为对应的到期收益率的折旧因子
    
    for i in range(len(Tenor)):
        if Tenor[i]<=1: #如果期限小于1，spot的折旧因子和到期收益率的折旧因子一致
            continue
        tenor_list=np.arange(Tenor[i]-1,0,-1)[::-1].round(5)#找到靴子的期限位置
        temp=np.isin(Tenor,tenor_list)
        before_period_flow = np.sum(discounted_factor[temp]*Par[i])#计算最后一期期之前的付息折现
        
        # 最终的折现因子=（全价-最后一期之前的付息的折现）/（1+c)
        discounted_factor[i] = (1+interest[i]- before_period_flow)/(1+Par[i])
    result = ((1/discounted_factor)**(1/Tenor)-1)*100 
    return(result)

In [188]:
Tenor=np.arange(0.1,10,0.1)
Par = Tenor*0.1+3
par_to_spot(Par = Par, Tenor = Tenor)

array([3.01      , 3.02      , 3.03      , 3.04      , 3.05      ,
       3.06      , 3.07      , 3.08      , 3.09      , 3.1       ,
       3.1062366 , 3.11387   , 3.12259676, 3.13220047, 3.14252274,
       3.15344503, 3.16487697, 3.17674854, 3.18900468, 3.20160159,
       3.20973023, 3.21847976, 3.22778045, 3.23757437, 3.24781307,
       3.25845572, 3.2694677 , 3.28081953, 3.29248596, 3.30444525,
       3.31333358, 3.32262473, 3.33229037, 3.34230561, 3.35264857,
       3.36329992, 3.37424255, 3.38546131, 3.39694271, 3.40867477,
       3.4180301 , 3.4276898 , 3.43763932, 3.44786558, 3.45835676,
       3.46910217, 3.48009216, 3.491318  , 3.50277177, 3.51444631,
       3.52415818, 3.5341197 , 3.54432249, 3.5547589 , 3.56542191,
       3.57630514, 3.58740273, 3.59870932, 3.61022004, 3.62193042,
       3.63195216, 3.64218953, 3.6526374 , 3.66329103, 3.6741461 ,
       3.68519861, 3.69644489, 3.70788161, 3.71950568, 3.73131432,
       3.74162823, 3.75213519, 3.76283198, 3.77371563, 3.78478

In [189]:
Par

array([3.01, 3.02, 3.03, 3.04, 3.05, 3.06, 3.07, 3.08, 3.09, 3.1 , 3.11,
       3.12, 3.13, 3.14, 3.15, 3.16, 3.17, 3.18, 3.19, 3.2 , 3.21, 3.22,
       3.23, 3.24, 3.25, 3.26, 3.27, 3.28, 3.29, 3.3 , 3.31, 3.32, 3.33,
       3.34, 3.35, 3.36, 3.37, 3.38, 3.39, 3.4 , 3.41, 3.42, 3.43, 3.44,
       3.45, 3.46, 3.47, 3.48, 3.49, 3.5 , 3.51, 3.52, 3.53, 3.54, 3.55,
       3.56, 3.57, 3.58, 3.59, 3.6 , 3.61, 3.62, 3.63, 3.64, 3.65, 3.66,
       3.67, 3.68, 3.69, 3.7 , 3.71, 3.72, 3.73, 3.74, 3.75, 3.76, 3.77,
       3.78, 3.79, 3.8 , 3.81, 3.82, 3.83, 3.84, 3.85, 3.86, 3.87, 3.88,
       3.89, 3.9 , 3.91, 3.92, 3.93, 3.94, 3.95, 3.96, 3.97, 3.98, 3.99])

In [190]:
Tenor

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])